In [12]:
import requests as req 
from bs4 import BeautifulSoup, CData, Tag
import time
import tqdm
import pickle
import pandas as pd

In [13]:
# pd.set_option('display.max_columns',None)
# pd.set_option('display.max_rows',None)

In [14]:
sites = { 
    'Фонтанка.ру' : ['https://www.fontanka.ru/fontanka.rss', ''],                  
    'LENTA.RU' : ['https://lenta.ru/rss', ''],                  
    'TASS' : ['https://tass.ru/rss/v2.xml', ''],
    'ВЕДОМОСТИ' : ['https://www.vedomosti.ru/rss/news', '']           
}

In [15]:
sites

{'Фонтанка.ру': ['https://www.fontanka.ru/fontanka.rss', ''],
 'LENTA.RU': ['https://lenta.ru/rss', ''],
 'TASS': ['https://tass.ru/rss/v2.xml', ''],
 'ВЕДОМОСТИ': ['https://www.vedomosti.ru/rss/news', '']}

In [16]:
def parsing():
    data = { "data" : [] }
        
    for site in range(len(sites)):
        
        url = list(sites.values())[site][0]
                
        resp = req.get(url)

        soup = BeautifulSoup(resp.content, features="xml")

        tag_item = soup.find_all('item')

        for item in tag_item:
            data['data'].append({
                "title": item.title.text,
                "pubDate": item.pubDate.text,
                "site": list(sites.keys())[site],
            })

    return pd.DataFrame(data['data'])

In [17]:
df = parsing()
df.sample(8)

,title,pubDate,site
43,На Украине захотели лишить наград артистов-пре...,"Sat, 21 Jan 2023 10:19:00 +0300",LENTA.RU
516,Путин призвал уже в этом году решить проблему ...,"Wed, 18 Jan 2023 16:36:42 +0300",ВЕДОМОСТИ
266,"Сбербанк установил банкоматы в Ялте, Симферопо...","Sat, 21 Jan 2023 10:44:02 +0300",TASS
110,В Запорожской области рассказали о продвижении...,"Sat, 21 Jan 2023 01:10:00 +0300",LENTA.RU
385,Памфилова заявила о подготовке к выборам прези...,"Fri, 20 Jan 2023 12:19:11 +0300",ВЕДОМОСТИ
82,Раскрыты новые попытки Украины завербовать сот...,"Sat, 21 Jan 2023 05:14:28 +0300",LENTA.RU
459,В Кремле предостерегли Запад от поставок воору...,"Thu, 19 Jan 2023 12:55:36 +0300",ВЕДОМОСТИ
158,США призвали Украину воздержаться от наступлен...,"Fri, 20 Jan 2023 21:06:00 +0300",LENTA.RU


In [18]:
topics = pd.DataFrame(['Библиотека', 'Россия', 'Мир', 'Экономика', 'Интернет и СМИ',
       'Спорт', 'Культура', 'Из жизни', 'Силовые структуры',
       'Наука и техника', 'Бывший СССР', 'nan', 'Дом', 'Сочи', 'ЧМ-2014',
       'Путешествия', 'Ценности', 'Легпром', 'Бизнес', 'МедНовости',
       'Оружие', '69-я параллель', 'Культпросвет ', 'Крым'])

In [19]:
model = None

with (open('../../lenta.ru/models/knb_ppl_clf.pkl', 'rb')) as openfile:
    model = pickle.load(openfile)

In [20]:
predicted_topics = model.predict(df['title'])

In [21]:
df_news = pd.concat([pd.Series(predicted_topics), df], ignore_index=True, axis=1)

df_news = df_news.rename(columns={0: "category", 1: "title", 2: "pub_date", 3: "site"})

df_news['parsed_date'] = None
df_news['day_of_week'] = None
for index, row in enumerate(df_news['pub_date']):
    parsed = time.strptime(row, "%a, %d %b %Y %H:%M:%S %z")
    df_news['parsed_date'][index] = time.strftime('%Y-%m-%d %H:%M:%S', parsed)
    df_news['day_of_week'][index] = time.strftime('%a', parsed)

df_news.sample(8)

,category,title,pub_date,site,parsed_date,day_of_week
128,Россия,Врач назвал самый полезный перекус для пожилых...,"Fri, 20 Jan 2023 23:07:00 +0300",LENTA.RU,2023-01-20 23:07:00,Fri
258,Спорт,В Республике Алтай нашли больного корью иностр...,"Sat, 21 Jan 2023 11:04:06 +0300",TASS,2023-01-21 11:04:06,Sat
522,Россия,Путин назначил нового посла в Монголии,"Wed, 18 Jan 2023 15:26:35 +0300",ВЕДОМОСТИ,2023-01-18 15:26:35,Wed
97,Мир,Шольца обвинили в страхе перед Россией,"Sat, 21 Jan 2023 03:08:00 +0300",LENTA.RU,2023-01-21 03:08:00,Sat
331,Экономика,Минфин США: союзники Вашингтона готовы ввести ...,"Sat, 21 Jan 2023 10:08:01 +0300",ВЕДОМОСТИ,2023-01-21 10:08:01,Sat
251,Россия,Голикова назвала стабильной ситуацию на рынке ...,"Sat, 21 Jan 2023 11:19:52 +0300",TASS,2023-01-21 11:19:52,Sat
456,Наука и техника,В Екатеринбурге таможенники изъяли у мужчины п...,"Thu, 19 Jan 2023 13:12:22 +0300",ВЕДОМОСТИ,2023-01-19 13:12:22,Thu
66,Бывший СССР,Турки разозлились из-за скандального ролика ук...,"Sat, 21 Jan 2023 07:16:05 +0300",LENTA.RU,2023-01-21 07:16:05,Sat


In [22]:
import json
from kafka import KafkaProducer

producer = KafkaProducer(bootstrap_servers='localhost:9092')

def write_to_kafka(df):
    hm = {}
    for _, row in df.iterrows():
        hm["category"] = row["category"]
        hm["title"] = row["title"]
        hm["site"] = row["site"]
        hm["pub_date"] = row["parsed_date"]        
        hm["day_of_week"] = row["day_of_week"]
        producer.send('foobar', json.dumps(hm).encode('utf-8'))
        producer.flush()

write_to_kafka(df_news)